In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('final_data2.csv', index_col=0)
pd.set_option('display.max_colwidth', None)

In [4]:
df.label.value_counts()

label
pos    1404118
neg     217096
Name: count, dtype: int64

In [5]:
df['label_num'] = df.label.map({'pos':0, 'neg':1})

In [6]:
#defining X and y for use with COUNTVECTORIZER
X = df.tweet
y = df.label_num
print(X.shape)
print(y.shape)

(1621214,)
(1621214,)


In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1215910,)
(405304,)
(1215910,)
(405304,)


In [8]:
X_train.describe()

count     1215819
unique    1161005
top          user
freq         2232
Name: tweet, dtype: object

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

# instantiate the vectorizer
vect = CountVectorizer()

In [10]:
# vect.fit(X_train)
# X_train_dtm = vect.transform(X_train)

In [11]:
# Document - term - matrix(dtm)
X_train_dtm = vect.fit_transform(X_train.values.astype('U'))

In [12]:
X_test_dtm = vect.transform(X_test.values.astype('U'))
X_test_dtm

<405304x240504 sparse matrix of type '<class 'numpy.int64'>'
	with 4747311 stored elements in Compressed Sparse Row format>

In [13]:
# import and instantiate a Multinomial Naive Bayes model
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

In [14]:
# train the model using X_train_dtm (timing it with an IPython "magic command")
%time nb.fit(X_train_dtm, y_train)

CPU times: user 128 ms, sys: 25.5 ms, total: 153 ms
Wall time: 159 ms


MultinomialNB()

In [15]:
# make class predictions for X_test_dtm
y_pred_class = nb.predict(X_test_dtm)

In [16]:
# calculate accuracy of class predictions
from sklearn import metrics
metrics.accuracy_score(y_test, y_pred_class)

0.9194456506720882

In [17]:
# print the confusion matrix
metrics.confusion_matrix(y_test, y_pred_class)

array([[345401,   5512],
       [ 27137,  27254]])

In [18]:
# print message text for the false positives (pos incorrectly classified as neg)
X_test[y_pred_class > y_test]

380056                                      do not get me wrong still sick just a little better i really do not want to sweep or do any chore
734547                                     in ma own house now bored w8in oan the xbox steve be play be oan at 2 yass cod 4 n the usual party
147842                                                           user ugh i forget today be the anniversery of ian s death now i be depressed
645726     haha jus bin tell wah to revise finnally a bit late now thou be not it spose ill do some i do not reli sleep much neways be bakk x
333755       i be deep and sensitive apparently i feel neither deep nor sensitive right now still feel awful but try to sty positive there be
                                                                          ...                                                                
581932                ugh block exam make me feel sad i suck so much that if i get a report card for life i would get an e i fail at life lol
119127

In [19]:
# print message text for the false negatives (neg incorrectly classified as pos)
X_test[y_pred_class < y_test]

489553                                    i feel a if i be almost get a cold no wonder a i have be around enough people who have one recently
1417596                                                                                work on a home make shelf for my books sanding cutting
307218                                                          very sad and disappoint that user and user do not come meet me at the airport
664154                                                   i think it be lt 3 i do not misinterpret your feeling i be wrong about mine ouch url
25690                                                                                     user sorry to hear it reivanna chronic one i assume
                                                                          ...                                                                
623509                                             sooooo tired i want to go back to sleep but cannot i get to wait for my sistah s furniture
822470

In [20]:
nb.predict_proba(X_test_dtm)

array([[7.63492597e-01, 2.36507403e-01],
       [9.99920909e-01, 7.90909282e-05],
       [9.66284786e-01, 3.37152135e-02],
       ...,
       [9.66251942e-01, 3.37480580e-02],
       [5.53975463e-01, 4.46024537e-01],
       [9.98062732e-01, 1.93726754e-03]])

In [21]:
# calculate predicted probabilities for X_test_dtm (poorly calibrated)
y_pred_prob = nb.predict_proba(X_test_dtm)[:, 1]
y_pred_prob

array([2.36507403e-01, 7.90909282e-05, 3.37152135e-02, ...,
       3.37480580e-02, 4.46024537e-01, 1.93726754e-03])

In [22]:
# calculate AUC
metrics.roc_auc_score(y_test, y_pred_prob)

0.9139196743121873

In [23]:
# import and instantiate a logistic regression model
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()

In [24]:
# train the model using X_train_dtm
%time logreg.fit(X_train_dtm, y_train)

CPU times: user 53.5 s, sys: 15 s, total: 1min 8s
Wall time: 10.7 s


/Users/achintyajha/miniconda3/envs/ml/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [25]:
# make class predictions for X_test_dtm
y_pred_class = logreg.predict(X_test_dtm)

In [26]:
# calculate predicted probabilities for X_test_dtm (well calibrated)
y_pred_prob = logreg.predict_proba(X_test_dtm)[:, 1]
y_pred_prob

array([4.10618898e-01, 2.66963959e-05, 2.93024116e-02, ...,
       5.63071711e-03, 8.12089074e-01, 1.07415046e-03])

In [27]:
# calculate accuracy
metrics.accuracy_score(y_test, y_pred_class)

0.9540098296587253

In [ ]:
# calculate AUC
metrics.roc_auc_score(y_test, y_pred_prob)

In [ ]:
# print message text for the false positives (pos incorrectly classified as neg)
X_test[y_pred_class > y_test]

In [ ]:
# print message text for the false negatives (neg incorrectly classified as pos)
X_test[y_pred_class < y_test]